In [45]:
"""Imports"""
import selenium
from datetime import datetime
from time import sleep as wait
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

#####################################################################################

""" URL """
username = "NEARProtocol"
url = f'https://twitter.com/{username}'
#url = "https://twitter.com/search?q=solana%20AND%20bluechip&src=typed_query&f=top"

""" FILTERS """
stringys = ["caught"]
before_date = datetime(2022, 6, 30)

#####################################################################################

#Creating an instance of a webdriver
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)

#GENERATE THE REQUEST !
driver.get(url)

#wait till load
wait(20)
print("Page ready !")

#Create tweet ID's for already scraped tweets
tweet_ids = set()
tweet_data = []

#Check if I'm at the end of the page
last_pos = driver.execute_script("return document.body.scrollHeight")
#print(last_pos)

#Set scrolilng
scrolling = True
in_range = True

#Looping through tweets
while scrolling:
    #Define the tweet segment
    cards = driver.find_elements(By.CSS_SELECTOR, "article")
    
    for card in cards: 
        
        """ Function """
        try:
            #Elements
            #GET FULL XPATH AND FOLLOW THROUGH AFTER TAG "ARTICLE" 
            text = card.find_element(By.XPATH, './div/div/div/div[2]/div[2]/div[2]/div[1]').text
            responding = card.find_element(By.XPATH, './div/div/div/div[2]/div[2]/div[2]/div[2]').text
            handle = card.find_element(By.XPATH, './/*[contains(text(),"@")]').text
            #replies = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
            #retweets = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
            #likes = card.find_element_by_xpath('.//div[@data-testid="like"]').text
            #pinned = card.find_element_by_xpath('./div/div/div/div[1]/div/div/div/div/div[2]/div/div/div').text
            full_txt = text+responding
            
            #Sponsored tweets have no date
            date = card.find_element(By.XPATH, './/time').get_attribute('datetime')

            """STR date into python datetime object"""
            #date = date.rstrip(".000Z")
            date = date[:-5]
            date = date.replace("T", " ")
            date_obj = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
            #print(date_obj)
            
        #NoSuchElement error handling
        except NoSuchElementException:
            pass
        #StaleElementReferenceException error handling
        except StaleElementReferenceException:
            pass
        
        if any(s in text.lower() for s in stringys):
        
            if date_obj > before_date:
                #Create a tuple for the tweet
                tweet = (handle, date, full_txt)
                print(tweet)

                #Create the tweet DF
                tweet_id = ''.join(tweet)

                #Add only tweets not already seen
                if tweet_id not in tweet_ids:
                    #Add id & data
                    tweet_ids.add(tweet_id)
                    tweet_data.append(tweet)
            """       
            #This makes the bot skip pinned tweets; Because they have dates which are late...
            if "Tweet fijado".lower() in pinned.lower():
                if stringy in text:
                    #Create a tuple for the tweet
                    tweet = (handle, date, full_txt)

                    #Create the tweet DF
                    tweet_id = ''.join(tweet)

                    #Add only tweets not already seen
                    if tweet_id not in tweet_ids:
                        #Add id & data
                        tweet_ids.add(tweet_id)
                        tweet_data.append(tweet)
            """
        if date_obj < before_date:
            scrolling = False
            in_range = False
            print("tweet out of range")
            break

            
    scrolling_attempt = 0
    
    while in_range == True:
        #Finally adding pagination
        driver.execute_script("window.scrollBy(0,500)")
        wait(1)

        #Current position and comparison to check if I'm at the bottom
        curr_pos = driver.execute_script("return document.body.scrollHeight")
        #print("current position:")
        #print(curr_pos)
        
        if last_pos == curr_pos: 
            #If I scrolled and ended up in the same place //nothing loaded//
            scrolling_attempt +=1
            
            #End of scroll region
            if scrolling_attempt >= 5: 
                scrolling = False
                break
            else:
                print(f'{scrolling_attempt}^st Attempt to scroll. Break in 5th.')
                wait(2)
        else:
            last_pos = curr_pos
            break
    else:
        break
                
"""Results"""
print("---------------------------------------------------")
print(f'Amount of tweets collected: {len(tweet_data)}')
print("---------------------------------------------------")
for i in tweet_data: 
    print("---------0---------")
    print(i)

driver.quit()

C:\Users\Usuario\AppData\Local\Temp/ipykernel_16928/491738858.py:27: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


Page ready !
1^st Attempt to scroll. Break in 3rd.
2^st Attempt to scroll. Break in 3rd.
1^st Attempt to scroll. Break in 3rd.
1^st Attempt to scroll. Break in 3rd.
2^st Attempt to scroll. Break in 3rd.
1^st Attempt to scroll. Break in 3rd.
1^st Attempt to scroll. Break in 3rd.
1^st Attempt to scroll. Break in 3rd.
1^st Attempt to scroll. Break in 3rd.
('@NEARProtocol', '2022-07-05 16:46:22', 'Exploring Web3 communities with NEAR  \n\n#NEAR caught up with \n@EndlesssFM\n, \n@StackSports\n, \n@near_ua\n, \n@NEARbalkan\n and NEAR Kenya to discuss the evolution of communities in Web3.\n\nMeet the faces behind the projects in Part 1 of the series ')
1^st Attempt to scroll. Break in 3rd.
2^st Attempt to scroll. Break in 3rd.
1^st Attempt to scroll. Break in 3rd.
tweet out of range
---------------------------------------------------
Amount of tweets collected: 1
---------------------------------------------------
---------0---------
('@NEARProtocol', '2022-07-05 16:46:22', 'Exploring Web3 co